In [1]:
import scanpy as sc
import pandas as pd
import scipy.sparse 
import numpy as np
from glob import glob
from pathlib import Path
import re
from multiprocessing import Pool

In [2]:
!cat "../../data/01_raw_counts/Metadata_HCL_Pateints.txt"

P1, P2, P3 Short_Responder
T0, T1, T2 Timepoints of sampling: TO = initial Diagnosis, T1 = 1. Relapse , T2 0 2. Relapse

P4, P5, P6 Long_Responder, Only T0 = Initial Diagnosis 

In [3]:
filenames = glob("../../data/01_raw_counts/*.csv")

In [4]:
filenames

['../../data/01_raw_counts/HCL_P1_T2.csv',
 '../../data/01_raw_counts/HCL_P2_T2.csv',
 '../../data/01_raw_counts/HCL_P4_T0.csv',
 '../../data/01_raw_counts/HCL_P3_T0.csv',
 '../../data/01_raw_counts/HCL_P1_T0.csv',
 '../../data/01_raw_counts/HCL_P3_T2.csv',
 '../../data/01_raw_counts/HCL_P2_T0.csv',
 '../../data/01_raw_counts/HCL_P1_T1.csv',
 '../../data/01_raw_counts/HCL_P2_T1.csv',
 '../../data/01_raw_counts/HCL_P5_T0.csv',
 '../../data/01_raw_counts/HCL_P6_T0.csv',
 '../../data/01_raw_counts/HCL_P3_T1.csv']

In [5]:
def load_counts(filename):
    p = Path(filename)
    sample = p.stem
    ((patient, timepoint), ) = re.findall("(P\d+)_(T\d+)\.csv", p.name)
    expr = pd.read_csv(p, skiprows=5, index_col=0)
    gene_expr = expr.loc[(~expr.index.str.contains("\(Ab\)") & ~expr.index.str.startswith("Lex_")), :]
    ab_expr = expr.loc[expr.index.str.contains("\(Ab\)"), :]
    obs = pd.DataFrame().assign(cell_id=expr.columns)
    obs["patient"] = patient
    obs["timepoint"] = timepoint
    obs["sample"] = sample
    obs.set_index("cell_id", inplace=True)
    adata = sc.AnnData(X=scipy.sparse.csc_matrix(gene_expr).T, obs=obs)
    adata.var_names = gene_expr.index
  #   adata.obsm["surface_protein"] = ab_expr.T
    return adata

In [6]:
with Pool(16) as p:
    adatas = p.map(load_counts, filenames)

In [7]:
sample_names = [Path(f).stem for f in filenames]

In [8]:
for adata, sample in zip(adatas, sample_names):
    adata.obs["sample"] = sample

In [9]:
for adata, sample in zip(adatas, sample_names):
    adata.write_h5ad(f"../../data/10_prepare_adata/{sample}.h5ad")

... storing 'patient' as categorical
... storing 'timepoint' as categorical
... storing 'sample' as categorical
... storing 'patient' as categorical
... storing 'timepoint' as categorical
... storing 'sample' as categorical
... storing 'patient' as categorical
... storing 'timepoint' as categorical
... storing 'sample' as categorical
... storing 'patient' as categorical
... storing 'timepoint' as categorical
... storing 'sample' as categorical
... storing 'patient' as categorical
... storing 'timepoint' as categorical
... storing 'sample' as categorical
... storing 'patient' as categorical
... storing 'timepoint' as categorical
... storing 'sample' as categorical
... storing 'patient' as categorical
... storing 'timepoint' as categorical
... storing 'sample' as categorical
... storing 'patient' as categorical
... storing 'timepoint' as categorical
... storing 'sample' as categorical
... storing 'patient' as categorical
... storing 'timepoint' as categorical
... storing 'sample' as cate